In [2]:
import pandas as pd
import zipfile

# Load CSV directly from zip file
with zipfile.ZipFile(r'C:\Users\Rayan\Desktop\Main Project\rsna-intracranial-aneurysm-detection.zip', 'r') as zip_ref:
    with zip_ref.open('train.csv') as file:
        df = pd.read_csv(file)

In [3]:
df.head()

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
0,1.2.826.0.1.3680043.8.498.10004044428023505108...,64,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,76,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,0,0,0,0,0,0,1,1
3,1.2.826.0.1.3680043.8.498.10009383108068795488...,71,Male,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.8.498.10012790035410518400...,48,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
feature_names = df.columns.tolist()
print(feature_names)

['SeriesInstanceUID', 'PatientAge', 'PatientSex', 'Modality', 'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery', 'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery', 'Left Middle Cerebral Artery', 'Right Middle Cerebral Artery', 'Anterior Communicating Artery', 'Left Anterior Cerebral Artery', 'Right Anterior Cerebral Artery', 'Left Posterior Communicating Artery', 'Right Posterior Communicating Artery', 'Basilar Tip', 'Other Posterior Circulation', 'Aneurysm Present']


In [5]:
df['Modality'].unique()

array(['MRA', 'CTA', 'MRI T2', 'MRI T1post'], dtype=object)

In [6]:
import pandas as pd
import zipfile

# Load train_localizer.csv directly from zip file
with zipfile.ZipFile(r'C:\Users\Rayan\Desktop\Main Project\rsna-intracranial-aneurysm-detection.zip', 'r') as zip_ref:
    with zip_ref.open('train_localizers.csv') as file:
        df_localizer = pd.read_csv(file)

# Print feature names
feature_names = df_localizer.columns.tolist()
print("Feature names in train_localizer.csv:")
print(feature_names)

# Optional: Print with numbering
print("\nNumbered feature list:")
for i, feature in enumerate(feature_names, 1):
    print(f"{i}. {feature}")

print(f"\nTotal number of features: {len(feature_names)}")
print(f"Dataset shape: {df_localizer.shape}")

Feature names in train_localizer.csv:
['SeriesInstanceUID', 'SOPInstanceUID', 'coordinates', 'location']

Numbered feature list:
1. SeriesInstanceUID
2. SOPInstanceUID
3. coordinates
4. location

Total number of features: 4
Dataset shape: (2251, 4)


<h1 color='blue'>Loading train and localization </h1>
    <p>extracting specific information if the anyuresum present more than one location</p>

In [7]:
import pandas as pd
import zipfile

# Load both CSV files from zip
with zipfile.ZipFile(r'C:\Users\Rayan\Desktop\Main Project\rsna-intracranial-aneurysm-detection.zip', 'r') as zip_ref:
    # Load train.csv
    with zip_ref.open('train.csv') as file:
        df_train = pd.read_csv(file)
    
    # Load train_localizer.csv
    with zip_ref.open('train_localizers.csv') as file:
        df_localizer = pd.read_csv(file)



Modality = CTA

In [8]:
# Filter for CTA modality and aneurysms in more than one location (from previous code)
aneurysm_columns = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery', 
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation'
]

# Filter for CTA modality
cta_data = df_train[df_train['Modality'] == 'CTA'].copy()

# Count aneurysms per instance
cta_data['aneurysm_count'] = cta_data[aneurysm_columns].sum(axis=1)

# Filter for instances with aneurysms in more than one location
multiple_aneurysms = cta_data[cta_data['aneurysm_count'] > 1]

# Merge with localizer data using SeriesInstanceUID
merged_data = pd.merge(cta_data[['SeriesInstanceUID', 'aneurysm_count']], 
                      df_localizer, 
                      on='SeriesInstanceUID', 
                      how='inner')

# Display the results
print("SeriesInstanceUID with CTA modality and multiple aneurysm locations:")
print("="*80)

for series_id in merged_data['SeriesInstanceUID'].unique():
    # Get data for this specific SeriesInstanceUID
    series_data = merged_data[merged_data['SeriesInstanceUID'] == series_id]
    aneurysm_count = series_data['aneurysm_count'].iloc[0]
    
    print(f"\nSeriesInstanceUID: {series_id}")
    print(f"Number of aneurysm locations: {aneurysm_count}")
    print(f"Number of localizer entries: {len(series_data)}")
    print("-" * 60)
    
    # Display each localizer entry for this series
    for idx, row in series_data.iterrows():
        print(f"  SOPInstanceUID: {row['SOPInstanceUID']}")
        print(f"  Coordinates: {row['coordinates']}")
        print(f"  Location: {row['location']}")
        print()

print(f"\nSummary:")
print(f"Total unique SeriesInstanceUIDs with multiple aneurysms: {len(multiple_aneurysms)}")
print(f"Total localizer entries for these series: {len(merged_data)}")

SeriesInstanceUID with CTA modality and multiple aneurysm locations:

SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317
Number of aneurysm locations: 1
Number of localizer entries: 1
------------------------------------------------------------
  SOPInstanceUID: 1.2.826.0.1.3680043.8.498.10775329348174902199350466348663848346
  Coordinates: {'x': 258.3621186176837, 'y': 261.359900373599}
  Location: Other Posterior Circulation


SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.10022796280698534221758473208024838831
Number of aneurysm locations: 1
Number of localizer entries: 1
------------------------------------------------------------
  SOPInstanceUID: 1.2.826.0.1.3680043.8.498.53868409774237283281776807176852774246
  Coordinates: {'x': 194.87253141831238, 'y': 178.32675044883302}
  Location: Right Middle Cerebral Artery


SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.10030095840917973694487307992374923817
Number of aneurysm locations: 1
Number of localize

Modality MRA

In [9]:
# Filter for CTA modality and aneurysms in more than one location (from previous code)
aneurysm_columns = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery', 
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation'
]

# Filter for CTA modality
cta_data = df_train[df_train['Modality'] == 'MRA'].copy()

# Count aneurysms per instance
cta_data['aneurysm_count'] = cta_data[aneurysm_columns].sum(axis=1)

# Filter for instances with aneurysms in more than one location
multiple_aneurysms = cta_data[cta_data['aneurysm_count'] > 1]

# Merge with localizer data using SeriesInstanceUID
merged_data = pd.merge(cta_data[['SeriesInstanceUID', 'aneurysm_count']], 
                      df_localizer, 
                      on='SeriesInstanceUID', 
                      how='inner')

# Display the results
print("SeriesInstanceUID with CTA modality and multiple aneurysm locations:")
print("="*80)

for series_id in merged_data['SeriesInstanceUID'].unique():
    # Get data for this specific SeriesInstanceUID
    series_data = merged_data[merged_data['SeriesInstanceUID'] == series_id]
    aneurysm_count = series_data['aneurysm_count'].iloc[0]
    
    print(f"\nSeriesInstanceUID: {series_id}")
    print(f"Number of aneurysm locations: {aneurysm_count}")
    print(f"Number of localizer entries: {len(series_data)}")
    print("-" * 60)
    
    # Display each localizer entry for this series
    for idx, row in series_data.iterrows():
        print(f"  SOPInstanceUID: {row['SOPInstanceUID']}")
        print(f"  Coordinates: {row['coordinates']}")
        print(f"  Location: {row['location']}")
        print()

print(f"\nSummary:")
print(f"Total unique SeriesInstanceUIDs with multiple aneurysms: {len(multiple_aneurysms)}")
print(f"Total localizer entries for these series: {len(merged_data)}")

SeriesInstanceUID with CTA modality and multiple aneurysm locations:

SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.10023411164590664678534044036963716636
Number of aneurysm locations: 1
Number of localizer entries: 1
------------------------------------------------------------
  SOPInstanceUID: 1.2.826.0.1.3680043.8.498.24186535344744886473554579401056227253
  Coordinates: {'x': 189.23979878597123, 'y': 209.19184886465828}
  Location: Right Middle Cerebral Artery


SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.10042423585566957032411171949972906248
Number of aneurysm locations: 1
Number of localizer entries: 1
------------------------------------------------------------
  SOPInstanceUID: 1.2.826.0.1.3680043.8.498.63062558671948377310688873437936911318
  Coordinates: {'x': 140.1086940862857, 'y': 177.63838968827815}
  Location: Right Middle Cerebral Artery


SeriesInstanceUID: 1.2.826.0.1.3680043.8.498.10058383541003792190302541266378919328
Number of aneurysm locations: 1
Number of local

In [11]:


# Distribution by aneurysm count (detailed)
print(f"\nDETAILED ANEURYSM COUNT DISTRIBUTION:")
aneurysm_dist = cta_data['aneurysm_count'].value_counts().sort_index()
for count, freq in aneurysm_dist.items():
    print(f"  {count} aneurysm(s): {freq} cases ({freq/len(cta_data)*100:.1f}%)")


DETAILED ANEURYSM COUNT DISTRIBUTION:
  0 aneurysm(s): 697 cases (55.7%)
  1 aneurysm(s): 492 cases (39.3%)
  2 aneurysm(s): 52 cases (4.2%)
  3 aneurysm(s): 9 cases (0.7%)
  4 aneurysm(s): 1 cases (0.1%)
  5 aneurysm(s): 1 cases (0.1%)
